<a href="https://colab.research.google.com/github/Oh-Seokjin/MRC_goorm/blob/main/utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Goorm/project02/kobigbird_ep10_max2048_lr6e-05_406.csv', encoding='utf-8')

In [ ]:
len(dataset)

4008

In [ ]:
start_stop_words = ["'", '‘', '“', '"', "‘‘‘", "'''", "<", "《", "(", "[", "@"]
end_stop_words = ["'", "’", "”", '"', "'''", "’’’", "》", ">", ")", "]", "@"]

In [ ]:
for i in range(len(dataset)):
    if str(dataset['Predicted'][i]).strip()[0] not in start_stop_words:
        continue
    for end_token in end_stop_words:
        if type(dataset['Predicted'][i]) is str and dataset['Predicted'][i].endswith(end_token) and dataset['Predicted'][i][0] in start_stop_words:
            dataset['Predicted'][i] = dataset['Predicted'][i][:len(dataset['Predicted'][i])-len(end_token)]
    for start_token in start_stop_words:
        if type(dataset['Predicted'][i]) is str and dataset['Predicted'][i].startswith(start_token):
            dataset['Predicted'][i] = dataset['Predicted'][i][len(start_token):]

In [ ]:
print(type(dataset['Predicted'][31]) == str)

True


In [ ]:
dataset.to_csv('submission_bracket_removed.csv', index=False)

In [ ]:
original = pd.read_csv('/content/drive/MyDrive/Goorm/project02/data/kobigbird_ep10_max2048_lr6e-05_406.csv')
removed = pd.read_csv('submission_bracket_removed.csv')

In [ ]:
original

,Id,Predicted
0,d14cb73158624cf094c546d856fd3c80,NaN
1,906631384e91493ebe1c7f34aea6f241,독일 뒤셀도르프로
2,35e61dcb479643448a2cb7d326ae50a6,링크트인
3,075e761b370040cb9041eecd39afc27c,링크트인
4,e67ed38f3dd944be94d5b4c53731f334,마드리드
...,...,...
4003,05fcb8054dc44dab8683579c2cf5e465,400만엔에서 200만엔
4004,cc7f826b66724ce9b39e3a974ca15661,중동 건설 현장
4005,3282034aa41e4fab980851ffd4a868dd,아시아~유럽 노선
4006,0a73550b36df4baf82ac2f98619d22e7,10일


In [ ]:
cnt = 0

for i in range(len(removed)):
    if original['Predicted'][i] != removed['Predicted'][i]:
        cnt += 1
        print(original['Predicted'][i], '  ---->  ', removed['Predicted'][i])

print(cnt)

nan   ---->   nan
‘운영허가 인증서’   ---->   운영허가 인증서
nan   ---->   nan
‘로사다운자켓’   ---->   로사다운자켓
‘충주 2차 푸르지오’   ---->   충주 2차 푸르지오
‘아베크 변주곡’   ---->   아베크 변주곡
nan   ---->   nan
nan   ---->   nan
《대전망 2014》   ---->   대전망 2014
nan   ---->   nan
‘법률소비자연맹’   ---->   법률소비자연맹
nan   ---->   nan
nan   ---->   nan
'동물'   ---->   동물
‘콘테스트 포스트시즌’   ---->   콘테스트 포스트시즌
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
‘빨대족’   ---->   빨대족
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
'황제'   ---->   황제
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
‘아크로힐스 논현’   ---->   아크로힐스 논현
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
nan   ---->   nan
‘파로스’   ---->   파로스
nan   ---->   nan


### Vote

In [ ]:
import pandas as pd
from pandas import Series, DataFrame

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''
1.  read_csv(비어있는 파일 생성 및 기존 결과)
    3개의 submission으로 진행한다고 가정

2.  hard vote 결과에 따라 최종 답안 결정
    a. 세 데이터셋의 답안이 같은 경우는 그대로
    b. 2:1인 경우 2표인 답안으로 결정
    c. 1:1:1인 경우 가장 점수가 높았던 제출본의 답안으로 결정
'''

submission1 = pd.read_csv('/content/drive/MyDrive/Goorm/project02/data/kobigbird_ep10_max2048_lr4e-05_batchsize_128.csv', encoding='utf-8') # 가장 높은 제출본
submission2 = pd.read_csv('/content/drive/MyDrive/Goorm/project02/data/kobigbird_ep10_max2048_lr6e-05_406.csv', encoding='utf-8')
submission3 = pd.read_csv('/content/drive/MyDrive/Goorm/project02/data/kobigbird_ep30_max2048_lr5e-05_4.csv', encoding='utf-8')


final_submission = DataFrame(submission3['Id'])
final_submission.insert(1, 'Predicted', ' ')

ensemble_predictions = []

for i in range(4008): # 테스트셋 크기에 따라 넣어서 사용, max 2773
    # 딕셔너리에 넣어서, 값이 있으면 그 값에 vote해주고, 없으면 새로 생성
    temp_pool = {}

    temp_pool[submission1['Predicted'][i]] = 1

    if submission2['Predicted'][i] in temp_pool:
        temp_pool[submission2['Predicted'][i]] = temp_pool[submission2['Predicted'][i]] +1
    else:
        temp_pool[submission2['Predicted'][i]] = 1

    if submission3['Predicted'][i] in temp_pool:
        temp_pool[submission3['Predicted'][i]] = temp_pool[submission3['Predicted'][i]] +1
    else:
        temp_pool[submission3['Predicted'][i]] = 1
    
    temp_pool = sorted(temp_pool.items(), key = lambda item: item[1], reverse=True)

    # 모두 같은 경우
    if len(temp_pool) == 1:
        ensemble_predictions.append(temp_pool[0][0])
    # 2:1인 경우
    elif len(temp_pool) == 2: 
        ensemble_predictions.append(temp_pool[0][0])
    # 1:1:1인 경우
    elif len(temp_pool) == 3:
        ensemble_predictions.append(submission1['Predicted'][i]) # 가장 점수 높은 제출본의 답안으로 결정

final_submission['Predicted'] = ensemble_predictions

final_submission.to_csv('submission_ensemble.csv', index=False)

In [ ]:
len(ensemble_predictions)

4008

In [ ]:
'34.7%' in temp_pool

True

In [ ]:
temp_pool['34.7%'] += 1

In [ ]:
temp_pool

In [ ]:
'''
1. 길이가 평가지표이므로, 결과 길이의 상한선을 정해서 제한하는 것도 한 방법
2. 품사 확인 후 의미에 영향 주지 않는 부분 제거하는 방법도 시도도
'''